# Wissensaggregator Mittelalter und frühe Neuzeit

WIAGweb2  
Daten für Utrechter Priester aus Excel einlesen

[Neuen Typ anlegen](#Neuen-Typ-anlegen)  
[Literatur](#Literatur)    
Priester  
- [Einträge in `item`](#Einträge-in-item)
- [Daten eintragen](#Daten-eintragen)
- [Freie Attribute](#Freie-Attribute)
- [Literaturverweise eintragen](#Literaturverweise-eintragen)
- [Namensvarianten eintragen](#Namensvarianten-eintragen)
- [Suchvarianten eintragen (`name_lookup`)](#Suchvarianten-eintragen-(name_lookup))

[Ämter](#Ämter)

## Vorspann

Modul laden

In [1]:
wds_path = "../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise

In [6]:
using WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1423


In [7]:
Wds = WiagDataSetup

WiagDataSetup

In [8]:
using DataFrames, CSV, Dates

In [9]:
ENV["COLUMNS"] = 120 # Anzeige/Ausgabe für die Inhalte von DataFrames vergrößern

120

In [10]:
computer = :bkroege_goe
computer = :ghertko_kn
if computer == :bkroege_goe
    data_path = "../data"
    out_path = "../data_sql"
else
    data_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data"
    out_path = "C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"
end

"C:/Users/georg/Documents/projekte-doc/Utrecht-Priester/data_sql"

In [11]:
item_type_id = 10

10

## Daten löschen
z.B. für ein komplettes Update

In [ ]:
sql_item_ids = "SELECT ID FROM item WHERE item_type_id = $(item_type_id)"
sql = "
DELETE FROM person_role WHERE person_id in ($(sql_item_ids));
DELETE FROM name_lookup WHERE person_id in ($(sql_item_ids));
DELETE FROM person WHERE id in ($(sql_item_ids));
DELETE FROM item_property WHERE item_id in ($(sql_item_ids));
DELETE FROM item_reference WHERE item_id in ($(sql_item_ids));
DELETE FROM item WHERE item_type_id = $(item_type_id);
"
file_name = joinpath(out_path, "delete_item_type_$(item_type_id).sql")
open(file_name, "w") do io
    println(io, sql)
end

## Neuen Typ anlegen

(einmalig)

einlesen aus CSV

In [ ]:
file_name = joinpath(data_path, "item_type.csv");
df_item_type = CSV.read(file_name, DataFrame, delim = ';')

In [ ]:
create_date(date_time) = typeof(date_time) == DateTime ? date_time : DateTime(date_time, dateformat"dd.mm.yy HH:MM")

In [ ]:
transform!(df_item_type, :date_created => ByRow(create_date) => :date_created);

In [ ]:
transform!(df_item_type, :date_changed => ByRow(create_date) => :date_changed);

In [ ]:
names(df_item_type)

alternativ: direkt erstellen

In [ ]:
df_item_type = DataFrame();

In [ ]:
insertcols!(df_item_type,
    :id => item_type_id,
    :comment => missing,
    :name => "Priester Utrecht",
    :note => "Priester des Bistums Utrecht nach Stapel",
    :table_name => "person",
    :id_public_key => "Pers-PRIESTUtrecht",
    :name_app => "priest_ut",
    :created_by => 2,
    :date_created => now(),
    :changed_by => 2,
    :date_changed => now(),
)

In [ ]:
file_name = joinpath(out_path, "item_type.sql")
Wds.insert_sql(file_name, "item_type", df_item_type)

## Literatur

In [ ]:
df_reference_volume = DataFrame();

In [ ]:
insertcols!(df_reference_volume,
    :id => missing, # wird automatisch vergeben
    :comment => missing,
    :full_citation => "Stapel, R.J. (2020) Ordinations of priests in the medieval diocese of Utrecht (1505-1518). IISH Data Collection.",
    :title_short => "Stapel, Ordinations of priests in the medieval diocese of Utrecht (1505-1518)",
    :author_editor => "Rombert Stapel",
    :note => missing,
    :ri_opac_id => missing,
    :isbn => missing,
    :year_publication => "2020",
    :display_order => 1,
    :gs_volume_nr => missing,
    :gs_doi => missing,
    :gs_url => missing,
    :online_resource => "https://hdl.handle.net/10622/IEYIUX",
    :reference_id => 1,
    :item_type_id => item_type_id
)

In [ ]:
file_name = joinpath(out_path, "reference_volume.sql")
Wds.insert_sql(file_name, "reference_volume", df_reference_volume)

## Einträge in `item`

Daten für die Priester einlesen.

Exportiere die Daten aus Excel über "Speichern unter" im Format "CSV UTF-8" (Auswahlfeld Dateityp)

In [ ]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [ ]:
Wds.clean_up!(df_priests)

In [ ]:
size(df_priests, 1)

In [ ]:
df_priests = unique(df_priests);

Der Datensatz einhält Zeilen, die in allen Feldern übereinstimmen.

In [ ]:
size(df_priests, 1)

In [ ]:
names(df_priests)

In [ ]:
columns = [
    :Priester_ID, 
    :Datum, 
    :Titel, 
    Symbol("Vorname (Voornaam)"), 
    Symbol("(Familienname) Familienaam")
]

In [ ]:
df_priests[100:105, columns]

In [ ]:
columns = [
    :Priester_ID => :id_in_source,
]

In [ ]:
df_item = select(df_priests, columns);

Verwende nur den Nummern-Teil der ID für die WIAG-ID

In [ ]:
get_id_num(id_long) = replace(id_long, "id_" => "")

In [ ]:
transform!(df_item, :id_in_source => ByRow(get_id_num) => :id_num_in_source);

In [ ]:
maximum(length.(df_item.id_num_in_source))

In [ ]:
id_public_key = df_item_type[1, :id_public_key]

In [ ]:
function make_id_public(id)        
    num_id_length = 10
    return "WIAG-" * id_public_key * "-" * lpad(id, num_id_length, '0')
end

In [ ]:
transform!(df_item, :id_num_in_source => ByRow(make_id_public) => :id_public);

In [ ]:
df_item[200:205, :]

Der Wert für `lang` ist das Kürzel für die Sprache der Quelle also `nl`?!

In [ ]:
insertcols!(df_item,
    :id => missing,
    :item_type_id => item_type_id,
    :comment => missing,
    :merged_into_id => missing,
    :edit_status => "online",
    :created_by => 2,
    :date_created => now(),
    :changed_by => 2,
    :date_changed => now(),
    :is_deleted => 0,
    :normdata_edited_by => missing,
    :lang => "nl",
    :is_online => 1
);

In [ ]:
df_item[200:205, :]

In [ ]:
table_name = "item"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, select(df_item, Not(:id_num_in_source)))

Importiere `item.sql` via phpMyAdmin in die Datenbank.  

## Daten eintragen

Exportiere die Tabelle `item` aus der Datenbank via phpMyAdmin mit den Optionen: "Spaltennamen in die erste Zeile setzen" und "Spalten trennen mit ';'" als Trennzeichen.

In [12]:
if computer == :bkroege_goe 
    file_name = joinpath(data_path, "item.csv");
else
    file_name = joinpath(data_path, "item_local.csv")
end
df_item_db = CSV.read(file_name, DataFrame, delim = ';');

In [13]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [14]:
Wds.clean_up!(df_priests)

In [15]:
size(df_priests, 1)

5255

In [16]:
df_priests = unique(df_priests);

In [17]:
df_item_utrecht = subset(df_item_db, :item_type_id => ByRow(isequal(item_type_id)));

In [18]:
size(df_item_utrecht, 1)

3958

In [19]:
df_idx = df_item_utrecht[:, [:id, :id_in_source, :item_type_id]];

In [20]:
names(df_priests)

11-element Vector{String}:
 "Priester_ID"
 "Datum"
 "Ort (Locatie)"
 "Titel"
 "Vorname (Voornaam)"
 "Zweiter Vorname (Patroniem)"
 "(Familienname) Familienaam"
 "Herkunftsort (Toponiem)"
 "Orden (Orde)"
 "Amt (Functie)"
 "Instelling"

In [21]:
columns = [
    :Priester_ID => :id_in_source,
    :Datum => :ordination_date,
    Symbol("Ort (Locatie)") => :ordination_place,
    :Titel => :title,
    Symbol("Vorname (Voornaam)") => :givenname,
    Symbol("Zweiter Vorname (Patroniem)") => :patronym,
    Symbol("(Familienname) Familienaam") => :familyname,
    Symbol("Herkunftsort (Toponiem)") => :place_of_origin,        
    Symbol("Orden (Orde)") => :religious_order,
    Symbol("Amt (Functie)") => :role_name,
    :Instelling => :institution
];

In [22]:
df_person_in = innerjoin(
    select(df_priests, columns),
    df_idx,
    on = :id_in_source
);

In [23]:
size(df_person_in, 1), size(df_priests, 1), size(df_idx, 1)

(3958, 3958, 3958)

### Identifiziere Orden

In [ ]:
file_name = joinpath(data_path, "religious_order.csv")
df_ro = CSV.read(file_name, DataFrame, delim = ";");

In [ ]:
df_item_ro = subset(df_item_db, :item_type_id => ByRow(isequal(7)));

In [ ]:
df_ro_id = innerjoin(df_ro, select(df_item_db, [:id, :item_type_id, :id_in_source]), on = :id);

Info

In [ ]:
df_ro_id

In [ ]:
file_name = joinpath(data_path, "mapping_orden_stapel.csv")
df_map_ro = CSV.read(file_name, DataFrame, delim = ";");

In [ ]:
transform!(df_map_ro, :id_order => ByRow(string) => :id_in_source);

In [ ]:
df_map_ro = innerjoin(df_map_ro, df_ro_id, on = :id_in_source);

In [ ]:
names(df_map_ro)

In [ ]:
columns = [
    :id => :religious_order_id,
    :id_in_source => :id_in_source_ro,
    :Orde => :religious_order,
    :abbreviation => :abbreviation,
]

In [ ]:
df_map_ro_join = select(df_map_ro, columns);

Die Ordensbezeichnungen variieren in Groß- und Kleinschreibung

In [ ]:
transform!(df_map_ro_join, :religious_order => ByRow(lowercase) => :religious_order);

In [ ]:
df_map_ro_join[7:12, :]

In [ ]:
lowercase_maybe(a) = typeof(a) <: AbstractString ? lowercase(a) : a

In [ ]:
transform!(df_person_in, :religious_order => ByRow(lowercase_maybe) => :religious_order);

In [ ]:
df_person_in_ro = leftjoin(df_person_in, df_map_ro_join, on = :religious_order, matchmissing = :notequal);

In [ ]:
size(df_person_in, 1), size(df_person_in_ro, 1)

In [ ]:
count(!ismissing, df_person_in.religious_order), count(!ismissing, df_person_in_ro.religious_order_id)

In [ ]:
delta_msg(a, b) = !ismissing(a) && ismissing(b)

In [ ]:
df_examine = subset(df_person_in_ro, [:religious_order, :religious_order_id] => ByRow(delta_msg));

In [ ]:
unique(df_examine.religious_order)

Exportiere Daten für die Tabelle `person`

In [ ]:
names(df_person_in_ro)

Verwende das Jahr der Priesterweihe für die zeitliche Einordnung

In [ ]:
parse_date_min(s) = ismissing(s) ? s : year(s)

In [ ]:
transform!(df_person_in_ro, :ordination_date => ByRow(parse_date_min) => :date_min);

In [ ]:
columns = [
    :id => :id,
    :item_type_id => :item_type_id,
    :givenname => :givenname,
    :familyname => :familyname,
    :religious_order_id => :religious_order_id,
    :date_min => :date_min,
    :date_min => :date_max,
];

In [ ]:
is_test(id) = id in (180293, 180301)

In [ ]:
subset(df_person_in_ro[:, [:id, :givenname, :date_min, :ordination_date, :title]], :id => ByRow(is_test))

In [ ]:
table_name = "person"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, select(df_person_in_ro, columns))

## Freie Attribute

### Titel

In [24]:
df_title = subset(df_person_in, :title => ByRow(!ismissing));

In [25]:
size(df_title)

(1712, 13)

In [26]:
df_item_prop_title = DataFrame();

In [27]:
insertcols!(df_item_prop_title,
    :item_id => df_title.id,
    :name => "title",
    :value => df_title.title,
    :date_value => missing,
);

In [28]:
df_item_prop_title[70:75, :]

,item_id,name,value,date_value
,Int64,String,String?,Missing
1,180293,title,Magister,missing
2,180295,title,Frater,missing
3,180296,title,Frater,missing
4,180297,title,Frater,missing
5,180299,title,Frater,missing
6,180301,title,Frater,missing


### Weihe Priester

Weiheamt

In [29]:
df_ordination = subset(df_person_in, :id => ByRow(!ismissing));
size(df_ordination)

(3958, 13)

In [30]:
df_item_prop_ord = DataFrame();

In [31]:
insertcols!(df_item_prop_ord,
    :item_id => df_ordination.id,
    :name => "ordination",
    :value => "Priester",
    :date_value => Date.(df_ordination.ordination_date)
);

Ort der Weihe

In [32]:
df_ord_place = subset(df_person_in, :ordination_place => ByRow(!ismissing));
size(df_ord_place)

(3958, 13)

In [33]:
df_item_prop_ord_place = DataFrame();

In [34]:
insertcols!(df_item_prop_ord_place,
    :item_id => df_ord_place.id,
    :name => "ordination_place",
    :value => df_ord_place.ordination_place,
    :date_value => missing,
);

Tabelle `item_property` schreiben

In [35]:
df_item_prop = vcat(df_item_prop_title, df_item_prop_ord, df_item_prop_ord_place);

In [ ]:
df_item_prop[1712:1714, :]

In [ ]:
size(df_item_prop)

In [36]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item_prop)

┌ Info: row 
│   i = 2000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1596
┌ Info: row 
│   i = 4000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1596
┌ Info: row 
│   i = 6000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1596
┌ Info: row 
│   i = 8000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1596


9628

### Geburtsort
2022-04-12: Trage zunächst nur den Namen ein. Die Zuordnung zu identifizierten Orten ist unklar.

In [ ]:
names(df_person_in)

In [ ]:
df_birthplace = subset(df_person_in, :place_of_origin => ByRow(!ismissing));
size(df_birthplace)

In [ ]:
df_item_prop_birthplace = DataFrame();

In [ ]:
insertcols!(df_item_prop_birthplace,
    :item_id => df_birthplace.id,
    :name => "birthplace",
    :value => df_birthplace.place_of_origin,
);

In [ ]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name)_birthplace.sql")
Wds.insert_sql(file_name, table_name, df_item_prop_birthplace)

## Literaturverweise eintragen
*R bk* Befülle `id_in_reference` mit 'id_1509_3_83' ... ?

In [ ]:
df_item_reference = DataFrame();

In [ ]:
insertcols!(df_item_reference,
    :reference_id => 1,
    :id_in_reference => df_person_in.id_in_source,
    :item_id => df_person_in.id,
    :item_type_id => item_type_id
);

In [ ]:
df_item_reference[1500:1505, :]

In [ ]:
table_name = "item_reference"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_item_reference)

## Namensvarianten eintragen
Schreibe Patronym (Vatersnamen) in Tabelle `item_property`

In [ ]:
df_patronym = subset(df_person_in, :patronym => ByRow(!ismissing));

In [ ]:
df_patronym[70:75, [:id, :givenname, :patronym]]

In [ ]:
df_item_prop_pym = DataFrame()

In [ ]:
insertcols!(df_item_prop_pym,
    :item_id => df_patronym.id,
    :name => "patronym",
    :value => df_patronym.patronym
);

In [ ]:
df_item_prop_pym[70:75, :]

In [ ]:
table_name = "item_property"
file_name = joinpath(out_path, "$(table_name)_patronym.sql")
Wds.insert_sql(file_name, table_name, df_item_prop_pym)

### Suchvarianten eintragen in `name_lookup`.
Das Patronym entspricht für die Suche am ehesten einem zweiten Vornamen.

In [ ]:
df_person_name = select(df_person_in, [:id, :givenname, :patronym, :familyname]);

In [ ]:
function join_space(a...)
    c_a = filter(!ismissing, collect(a))
    return join(c_a, " ")
end

In [ ]:
df_name_variant = transform(df_person_name, [:givenname, :patronym] => ByRow(join_space) => :givenname);

In [ ]:
df_name_variant[50:55, :]

Füge technisch erforderliche Spalten ein. Erzeuge Einträge für `name_lookup`.

In [ ]:
insertcols!(df_name_variant,
    :givenname_variant => missing,
    :familyname_variant => missing,
    :prefix_name => missing,
    :lang => "nl"
);

In [ ]:
df_nl = Wds.create_name_lookup(df_name_variant);

Info

In [ ]:
df_gerardus = subset(df_name_variant, :id => ByRow(isequal(203857)))

In [ ]:
df_nl[55:60, :]

In [ ]:
names(df_nl)

In [ ]:
table_name = "name_lookup"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_nl)

## Ämter

In [ ]:
file_name = joinpath(data_path, "item.csv");
df_item_db = CSV.read(file_name, DataFrame, delim = ';');

In [ ]:
file_name = joinpath(data_path, "Ordinations_of_priests_Stapel_BK.csv");
df_priests = CSV.read(file_name, DataFrame, delim = ';', stringtype = String);

In [ ]:
Wds.clean_up!(df_priests)

In [ ]:
size(df_priests, 1)

In [ ]:
df_priests = unique(df_priests);

In [ ]:
df_item_utrecht = subset(df_item_db, :item_type_id => ByRow(isequal(10)));

In [ ]:
size(df_item_utrecht, 1)

In [ ]:
columns = [
    :id => :person_id,
    :id_in_source => :id_in_source,
    :item_type_id => :item_type_id,
]

In [ ]:
df_idx = select(df_item_utrecht, columns);

In [ ]:
columns = [
    :Priester_ID => :id_in_source,
    :Datum => :ordination_date,
    Symbol("Ort (Locatie)") => :ordination_place,
    :Titel => :title,
    Symbol("Vorname (Voornaam)") => :givenname,
    Symbol("Zweiter Vorname (Patroniem)") => :patronym,
    Symbol("(Familienname) Familienaam") => :familyname,
    Symbol("Herkunftsort (Toponiem)") => :place_of_origin,        
    Symbol("Orden (Orde)") => :religious_order,
    Symbol("Amt (Functie)") => :role_name,
    :Instelling => :institution
];

In [ ]:
names(df_idx)

In [ ]:
df_priests_with_role = subset(select(df_priests, columns), :role_name => ByRow(!ismissing));

In [ ]:
df_person_role = innerjoin(
    df_priests_with_role,
    df_idx,
    on = :id_in_source
);

In [ ]:
size(df_person_role, 1), size(df_priests, 1), size(df_idx, 1)

Setze Diözese 'Utrecht', ID = 53

In [ ]:
insertcols!(df_person_role, :diocese_id => 53, :diocese_name => "Utrecht");

In [ ]:
df_person_role[30:35, [:id_in_source, :givenname, :familyname, :role_name, :diocese_name, :diocese_id]]

### Rollen zuordnen

In [ ]:
file_name = joinpath(data_path, "role.csv");
df_role = CSV.read(file_name, DataFrame);

In [ ]:
names(df_role)

In [ ]:
df_person_role = leftjoin(df_person_role, 
    select(df_role, [:id => :role_id, :name => :role_name_norm]), 
    on = :role_name => :role_name_norm);

In [ ]:
count(ismissing, df_person_role.role_id)

In [ ]:
columns = [
    :person_id, :id_in_source, :title, :givenname, :patronym, :familyname, :role_name, :role_id
];

In [ ]:
subset(select(df_person_role, columns), :role_id => ByRow(ismissing))

In [ ]:
names(df_person_role)

In [ ]:
columns = [
    :person_id => :person_id,
    :role_id => :role_id,
    :role_name => :role_name,
    :diocese_id => :diocese_id,
    :diocese_name => :diocese_name,
    :institution => :institution_name,
];

In [ ]:
df_person_role_out = select(df_person_role, columns);

In [ ]:
table_name = "person_role"
file_name = joinpath(out_path, "$(table_name).sql")
Wds.insert_sql(file_name, table_name, df_person_role_out)

Dateien zusammenfügen

In [ ]:
function cat_sql(file_out, file_in...; path = ".")    
    open(joinpath(path, file_out), "w") do io_out
        for file_loop in file_in            
            open(joinpath(path, file_loop)) do io_in
                write(io_out, read(io_in))
            end
        end
    end
end

Inhalte, die unabhängig sind von einzelnen Personen: `item_type.sql`, `reference_volume.sql`

Inhalte für die einzelnen Priester. (Tabelle `item` muss schon im Prozess geschrieben und gelesen worden sein).

In [ ]:
file_out = "item_x.sql"
cat_sql(file_out,
    "item_property.sql",
    "item_property_patronym.sql",
    "item_reference.sql",
    path = out_path,
)

In [ ]:
file_out = "person_x.sql"
cat_sql(file_out,
    "person.sql",
    "person_role.sql",
    "name_lookup.sql",
    path = out_path,
)